# Task C
## 1. Write a python “generate streams” application to feed data from a file into a stream. The application should set the timing such that one tuple from “Climate Data-Part2” and five tuple from “Fire Data-Part2” are sent into the stream every second. “DataType” column in the record can be used to distinguish between the streams.
## 2. Write a streaming application in Apache Spark with local Streaming Context with two execution threads and a batch interval of five seconds. The application will

###    a. Receive streaming data from the streams generated by the stream server.
###    b. Process the streams to ensure the data can be uploaded into the MongoDB according to the data model you have designed in Task B1.
###    c. Upload the result of processing in Task C2b to MongoDB.
## 3. Testing: Use the python application from Task B (as2TaskB.ipynb) to show thestreamed data have been added into the database successfully.

In [1]:
import csv
import pymongo
import uuid
import json
import socket
from pymongo import MongoClient
from pyspark import SparkContext # spark
from pyspark.streaming import StreamingContext # spark streaming


In [2]:
# read file data to memeory
# fireData
fireData = []
with open('FireData-Part2.csv') as firePath:
    reader = csv.reader(firePath)
    for row in reader:
        fireData.append(row)
#climateData
climateData = []
with open('ClimateData-Part2.csv') as climatePath:
    reader = csv.reader(climatePath)
    for row in reader:
        climateData.append(row)
del climateData[0]
del fireData[0]




In [3]:
#this function use to remove space for each entity and make each line list become a str
def formatData(dataset):
    newdataset = []
#remove space
    for eachRecordIndex in range(len(dataset)):
        for eachEntity in range(len(dataset[eachRecordIndex])):
            dataset[eachRecordIndex][eachEntity] = dataset[eachRecordIndex][eachEntity].strip()
#make each line list become a str and add each str to new list
    for eachRecord in dataset:
        line = ''
        for eachEntity in eachRecord:
            line = line + "," + eachEntity
        newdataset.append(line[1:])
    return newdataset
    

In [4]:
formatedClimate = formatData(climateData)
formatedFire = formatData(fireData)

In [5]:
# at here we look the length for climate data, the length represent how many time we need send climate data to server
len(formatedClimate)

366

In [6]:
# if every time we send 5 record to sever, the code can calculate how many time we need to send firedata
len(formatedFire)/5

533.6

In [7]:
#because 533.6 is more than 366, and the 533.6 shoild be 534. 534 is the whole time we send data to server
sendTime = int(len(formatedFire)/5) + 1
print(sendTime)


534


In [8]:
def generateStream(sendTime):
    for time in range(sendTime):
        try:
            yield formatedClimate[time:time+1], formatedFire[time*5:(time+1)*5]
        except:
            yield formatedFire[time*5:(time+1)*5]
stream = generateStream(sendTime)

In [9]:
def sendPartion(stream): 
    host = "localhost"
    port = 9999
    partion = json.dumps(next(stream)).encode()
    partion = partion.decode()
    climateBlock = json.loads(partion)[0]
    fireBlock = json.loads(partion)[1]
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.connect((host, int(port)))
    server.sendall(climateBlock,fireBlock)
    server.shutdown(socket.SHUT_WR)
    while True:
        data = s.recv(4092)
        if not data:
            break
        print(repr(data))
    s.close()